In [0]:
import pandas as pd 
import surprise as sur
from surprise.model_selection import cross_validate

Read in the review dataset and prepare the dataset to be processed by the Surprise library using the Reader and Dataset classes.

In [0]:
df = pd.read_csv('df_sub.csv.gz', 
                       compression='gzip').astype({'rating':'int8', 'total_votes':'int32'})

In [0]:
reader = sur.Reader(rating_scale=(0,5))
data = sur.Dataset.load_from_df(df[['reviewerId', 'asin','rating']], reader)

## Calculating RMSE for all algos in Surprise

I first wanted to find the best algorithms offered by the Surprise in terms of minimising RMSE. I used code from [this notebook](https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Building%20Recommender%20System%20with%20Surprise.ipynb) to iterate and cross validate all the algorithms to obtain RMSE, MAE, fit times and test times so I could pick the best ones to gridsearch even further.

After calculating the RMSE scores for all the algorithms, I decided to pick BaselineOnly, SVD and KNNBaseline to gridsearch further to optimise the RMSE through hypertuning. I decided to chose SVD over SVDpp as it performed only marginall better that SVD which wasn't enough to justify the fit time. These were done in seperate notebook as I used hosted runtime on Google Collabs to ease the load on my laptop.

In [0]:
benchmark = []
 
for algorithm in [sur.SVD(), sur.SVDpp(), sur.SlopeOne(), sur.NMF(), sur.NormalPredictor(), sur.KNNBaseline(), sur.KNNBasic(), sur.KNNWithMeans(), sur.KNNWithZScore(), sur.BaselineOnly(), sur.CoClustering()]:
    results = cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
final_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
final_results.to_csv('results_cv_recommnder.csv', header=True)
final_results

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8601  0.8606  0.8656  0.8621  0.0025  
MAE (testset)     0.6644  0.6657  0.6693  0.6665  0.0021  
Fit time          12.54   12.99   12.83   12.79   0.19    
Test time         0.95    1.29    1.09    1.11    0.14    
Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8599  0.8588  0.8585  0.8591  0.0006  
MAE (testset)     0.6624  0.6599  0.6613  0.6612  0.0010  
Fit time          363.73  345.33  351.35  353.47  7.66    
Test time         17.77   18.45   19.33   18.52   0.64    
Evaluating RMSE, MAE of algorithm SlopeOne on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8916  0.8952  0.8892  0.8920  0.0025  
MAE (testset)     0.6822  0.6823  0.6826  0.6824  0.0002  
Fit time          7.21    8.16    7.53    7.63    0.39    
Test 

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
BaselineOnly,0.855957,0.666846,1.051120,1.013781
SVDpp,0.859079,0.661183,353.472611,18.518866
SVD,0.862076,0.666452,12.785394,1.107035
KNNBaseline,0.868869,0.666314,1.774138,7.893475
KNNWithMeans,0.871516,0.666048,0.896398,7.516442
KNNWithZScore,0.875444,0.665424,1.186114,7.462321
CoClustering,0.891596,0.681144,6.491434,1.251335
SlopeOne,0.891981,0.682352,7.632689,13.265806
NMF,0.924378,0.716844,17.285174,0.939333
